# Gerar comparação entre palavras dos documentos originais das sentenças do Cohebert manual

Gera a comparação entre as palavras das sentenças dos documentos do conjunto de dados Cohebert manual utilizando os arquivos:
- `original.zip`
- `originalpos.zip`

Cria o arquivo `comparacao_palavra_pX_kY.zip` com as comparações entre as palavras do documento, onde X é o número de documentos perturbados e Y o valor de top K predições.




# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicioProcessamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """    
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documentoLimpo = re.compile("<.*?>")
    return re.sub(documentoLimpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nomeArquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nomeArquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nomeArquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nomeArquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [ ]:
def salvar(nomeArquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nomeArquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempoArredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempoArredondado))    

Classe(ModelArguments) de definição dos parâmetros do modelo

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    janela: int = field(
        default="0",
        metadata={"help": "Quantidade de sentenças que formam a janela de texto a ser submetido ao BERT."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )    
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )   
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    )

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## 1.4 Identificando o ambiente Colab

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))    
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

2022-03-09 12:17:36,563 : INFO : NumExpr defaulting to 2 threads.
2022-03-09 12:17:37,304 : INFO : Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def getDeviceGPU():
    """
    Retorna um dispositivo de GPU se disponível ou CPU.
    
    Retorno:
    `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [ ]:
device = getDeviceGPU()

2022-03-09 12:17:44,877 : INFO : Sem GPU disponível, usando CPU.


Conecta o modelo ao device

In [ ]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def conectaGPU(model, device):
    """
      Conecta um modelo BERT a GPU.

      Parâmetros:
        `model` - Um modelo BERT carregado.       
        `device` - Um device de GPU.     
    
      Retorno:
        `model` - Um objeto model BERT conectado a GPU.     
    """
    # Associa a GPU ao modelo.
    model.to(device)

    # Se existe GPU disponível.
    if torch.cuda.is_available():    
        # Diga ao pytorch para rodar este modelo na GPU.
        logging.info("Pytorch rodando o modelo na GPU.")
        model.cuda()
        
    else:
        logging.info("Pytorch rodando sem GPU.")

    return model

Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

2022-03-09 12:17:44,977 : INFO : Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

2022-03-09 12:17:44,989 : INFO : Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
2022-03-09 12:17:44,994 : INFO : e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")     

Mounted at /content/drive


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 144 kB 44.5 MB/s 
     |████████████████████████████████| 181 kB 47.6 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7828b15a407a6c6be7cf9de1b27dc2045b06675b73dab01a8f08113848405d65
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U pip setuptools wheel

     |████████████████████████████████| 2.1 MB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.0/452.0 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.2/628.2 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 62.7 MB/s eta 0:00:00
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installa

## 1.9 Instalação do BERT

Instala a interface pytorch para o BERT by Hugging Face. 

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

In [ ]:
!pip install -U transformers==4.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.2/895.2 KB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.4 MB/s eta 0:00:00


# 2 Parametrização

## Gerais

In [ ]:
# Definição dos parâmetros a serem avaliados
#Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 1 # Somente 1 para o Cohebert Manual

#Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 1 # Somente 1 para o Cohebert Manual

## Específicos

In [ ]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosMedida(     
    max_seq_len = 512, 
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",    
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",
    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",
    versao_modelo_spacy = "3.2.0",
    sentenciar_documento = True,
    janela = 2,
    do_lower_case = False,   # default True  
    output_attentions = False,    # default False
    output_hidden_states = True, # default False, se True retorna todas as camadas do modelo para as operações de soma e concatenação
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path.     
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara.
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHEBERT_MANUAL"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [ ]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
        
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [ ]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    
    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivoTar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")    
    arquivoTar.extractall(DIRETORIO_COHEBERT)    
    arquivoTar.close()
    
    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):        
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.
    
    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.           
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
                  
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp 

### Carrega o modelo spaCy


In [ ]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

2022-03-09 12:19:46,399 : INFO : Diretório Cohebert criado: COHEBERT_MANUAL
2022-03-09 12:19:46,404 : INFO : Download do arquivo do modelo do spaCy.
2022-03-09 12:19:47,332 : INFO : Download do arquivo: COHEBERT_MANUAL/pt_core_news_lg-3.2.0.tar.gz.


  0%|          | 0.00/577M [00:00<?, ?B/s]

2022-03-09 12:20:36,227 : INFO : Descompactando o arquivo do modelo do spaCy.
2022-03-09 12:20:51,061 : INFO : spaCy carregado.


## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [ ]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).
    
      Parâmetros:
        `nlp` - Um modelo spaCy carregado.           
    """
    
    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords 

Lista dos stopwords

In [ ]:
logging.info("Quantidade de stopwords: {}.".format(len(getStopwords(nlp))))

print(getStopwords(nlp))

2022-03-09 12:20:51,089 : INFO : Quantidade de stopwords: 416.


{'assim', 'foi', 'sempre', 'embora', 'do', 'povo', 'aquela', 'pelo', 'com', 'pôde', 'dos', 'favor', 'estiveste', 'podia', 'tão', 'minha', 'mil', 'mês', 'tiveste', 'fora', 'nesta', 'põe', 'naquele', 'pela', 'pelos', 'num', 'local', 'corrente', 'nessa', 'estou', 'fazes', 'possivelmente', 'esteve', 'já', 'vezes', 'de', 'são', 'bastante', 'conhecido', 'dois', 'dá', 'perto', 'dezoito', 'vocês', 'des', 'porém', 'cuja', 'ambos', 'nove', 'dão', 'quatro', 'muito', 'nosso', 'as', 'vinte', 'esses', 'depois', 'cá', 'para', 'obrigada', 'meu', 'essa', 'demais', 'veja', 'sei', 'quem', 'catorze', 'vos', 'fazem', 'eles', 'ver', 'estás', 'menos', 'tua', 'numa', 'além', 'sétima', 'tarde', 'vossa', 'qual', 'nunca', 'coisa', 'ou', 'nós', 'enquanto', 'sobre', 'daquele', 'sem', 'sexta', 'quando', 'está', 'breve', 'comprida', 'fará', 'menor', 'dez', 'segundo', 'lá', 'da', 'faço', 'deste', 'todo', 'boa', 'ontem', 'e', 'sistema', 'maior', 'das', 'meses', 'sexto', 'treze', 'tive', 'desde', 'fazia', 'neste', 'lug

### getVerbos
Localiza os verbos da sentença

In [ ]:
# Import das bibliotecas.
import spacy   
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                                  
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar   
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ] 

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [               
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                   
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal       
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ] 

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar 
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}}, 
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar 
               ] 

matcherv = Matcher(nlp.vocab)
         
matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):    
  #Processa o período
  doc1 = nlp(periodo.text)
  
  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))
  
  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = dict()
  
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

In [ ]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades    
  novodic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}
    
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

### getDicTodasPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicTodasPOSQtde(lista):

  # Dicionário com as tags e quantidades
  conjunto = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ": 0}

  for x in lista:
    valor = conjunto.get(x)
    if valor != None:
      conjunto[x] = valor + 1
    else:
      conjunto[x] = 1

  return conjunto

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [ ]:
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):
    
  # Soma os dicionários da lista
  novodic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))
 
  return novodic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [ ]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [ ]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [ ]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  listatokens = []
  listapos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:    
    listatokens.append(token.text)
    listapos.append(token.pos_)
    
  return listatokens, listapos

### Tadução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [ ]:
#dicionário que contêm pos tag universal e suas explicações
palavrauniversal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}
  
#Explica a POS
def traduzPOSPalavraUniversal(palavra):
  if palavra in palavrauniversal_dict.keys():
      traduzido = palavrauniversal_dict[palavra]
  else:
      traduzido = "NA" 
  return traduzido

### getSentencaSemStopWord

Retorna uma lista dos tokens sem as stopwords.

In [ ]:
def getSentencaSemStopWord(sentenca, stopwords):

  # Lista dos tokens
  lista = []

  # Percorre os tokens da sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é uma stopword
    if token.lower() not in stopwords:
      lista.append(token)

  # Retorna o documento
  return lista

### getSentencaSalientePOS

Retorna uma lista das palavras do tipo especificado.

In [ ]:
def getSentencaSalientePOS(sentenca, pos, tipoSaliente="NOUN"):
  
  # Lista dos tokens
  lista = []

  # Percorre a sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é do tipo especeficado
    if pos[i] == tipoSaliente:
      lista.append(token)

  # Retorna o documento
  return lista

###removeStopWords

Remove as stopwords de um documento ou senteça.

In [ ]:
def removeStopWord(documento, stopwords):
  
  # Remoção das stopwords do documento
  documentoSemStopwords = [palavra for palavra in documento.split() if palavra.lower() not in stopwords]

  # Concatena o documento sem os stopwords
  documentoLimpo = " ".join(documentoSemStopwords)

  # Retorna o documento
  return documentoLimpo

### retornaRelevante

Retorna somente os palavras do documento ou sentença do tipo especificado.

In [ ]:
def retornaRelevante(documento, classeRelevante="NOUN"):

  # Corrigir!
  # Utilizar o documento já tokenizado pelo spacy!!!!
  # Existe uma lista com o documento e a sentença tokenizada pelo spacy
  
  # Realiza o parsing no spacy
  doc = nlp(documento)

  # Retorna a lista das palavras relevantes
  documentoComSubstantivos = []
  for token in doc:
    #print("token:", token.pos_)
    if token.pos_ == classeRelevante:
      documentoComSubstantivos.append(token.text)

  # Concatena o documento com os substantivos
  documentoConcatenado = " ".join(documentoComSubstantivos)

  # Retorna o documento
  return documentoConcatenado

# 4 BERT

## 4.1 Modelo Pré-treinado BERT

### Funções Auxiliares

In [ ]:
def getNomeModeloBERT(model_args):
    '''    
    Recupera uma string com uma descrição do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `MODELO_BERT` - Nome do modelo BERT.
    '''

    # Verifica o nome do modelo(default SEM_MODELO_BERT)
    MODELO_BERT = "SEM_MODELO_BERT"
    
    if 'neuralmind' in model_args.pretrained_model_name_or_path:
        MODELO_BERT = "_BERTimbau"
        
    else:
        if 'multilingual' in model_args.pretrained_model_name_or_path:
            MODELO_BERT = "_BERTmultilingual"
            
    return MODELO_BERT

In [ ]:
def getTamanhoBERT(model_args):
    '''    
    Recupera uma string com o tamanho(dimensão) do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `TAMANHO_BERT` - Nome do tamanho do modelo BERT.
    '''
    
    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = "_large"
    
    if 'base' in model_args.pretrained_model_name_or_path:
        TAMANHO_BERT = "_base"
        
    return TAMANHO_BERT  

### Função download Modelo Pre-treinado BERT

In [ ]:
# Import das bibliotecas.
import zipfile # Biblioteca para descompactar
import shutil # iblioteca de manipulação arquivos de alto nível

def downloadModeloPretreinado(model_args):
    """
      Realiza o download do modelo BERT(MODELO) e retorna o diretório onde o modelo BERT(MODELO) foi descompactado.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    
      Retorno:
        `DIRETORIO_MODELO` - Diretório de download do modelo.
    """ 
    
    # Nome diretório base modelo BERT
    NOME_DIRETORIO_BASE_MODELO = "modeloBERT"
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Recupera o nome ou caminho do modelo
    MODELO = model_args.pretrained_model_name_or_path

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in MODELO:
        URL_MODELO = MODELO

    # Se a variável foi setada.
    if URL_MODELO:

        # Diretório do modelo.
        DIRETORIO_MODELO = DIRETORIO_COHEBERT + "/" + NOME_DIRETORIO_BASE_MODELO
        
        # Recupera o nome do arquivo do modelo da url.
        NOME_ARQUIVO = URL_MODELO.split("/")[-1]

        # Nome do arquivo do vocabulário.
        ARQUIVO_VOCAB = "vocab.txt"
        
        # Caminho do arquivo na url.
        CAMINHO_ARQUIVO = URL_MODELO[0:len(URL_MODELO)-len(NOME_ARQUIVO)]

        # Verifica se o diretório de descompactação existe no diretório corrente
        if os.path.exists(DIRETORIO_MODELO):
            logging.info("Apagando diretório existente do modelo!")
            # Apaga o diretório e os arquivos existentes                     
            shutil.rmtree(DIRETORIO_MODELO)
        
        # Realiza o download do arquivo do modelo        
        downloadArquivo(URL_MODELO, NOME_ARQUIVO)

        # Descompacta o arquivo no diretório de descompactação.                
        arquivoZip = zipfile.ZipFile(NOME_ARQUIVO, "r")
        arquivoZip.extractall(DIRETORIO_MODELO)

        # Baixa o arquivo do vocabulário.
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente.
        URL_MODELO_VOCAB = CAMINHO_ARQUIVO + ARQUIVO_VOCAB
        # Coloca o arquivo do vocabulário no diretório do modelo.        
        downloadArquivo(URL_MODELO_VOCAB, DIRETORIO_MODELO + "/" + ARQUIVO_VOCAB)
        
        # Apaga o arquivo compactado
        os.remove(NOME_ARQUIVO)

        logging.info("Diretório {} do modelo BERT pronta!".format(DIRETORIO_MODELO))

    else:
        DIRETORIO_MODELO = MODELO
        logging.info("Variável URL_MODELO não setada!")

    return DIRETORIO_MODELO

### Copia o modelo do BERT ajustado

In [ ]:
# Import das bibliotecas.
import shutil # iblioteca de manipulação arquivos de alto nível

def copiaModeloAjustado(model_args):
    """ 
      Copia o modelo ajustado BERT do GoogleDrive para o projeto.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    
      Retorno:
        `DIRETORIO_LOCAL_MODELO_AJUSTADO` - Diretório de download ajustado do modelo.
    """

    # Verifica o nome do modelo BERT a ser utilizado
    MODELO_BERT = getNomeModeloBERT(model_args)

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = getTamanhoBERT(model_args)

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Diretório local de salvamento do modelo.
    DIRETORIO_LOCAL_MODELO_AJUSTADO = DIRETORIO_COHEBERT + "/modelo_ajustado/"

    # Diretório remoto de salvamento do modelo no google drive.
    DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/holdout/modelo/" + MODELO_BERT + TAMANHO_BERT

    # Copia o arquivo do modelo para o diretório no Google Drive.
    shutil.copytree(DIRETORIO_REMOTO_MODELO_AJUSTADO, DIRETORIO_LOCAL_MODELO_AJUSTADO) 
   
    logging.info("Modelo BERT ajustado copiado!")

    return DIRETORIO_LOCAL_MODELO_AJUSTADO

### Verifica de onde utilizar o modelo do BERT

In [ ]:
def verificaModelo(model_args):
    """ 
    Verifica de onde utilizar o modelo.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.
    
    Retorno:
    `DIRETORIO_MODELO` - Diretório de download do modelo.
    """ 

    DIRETORIO_MODELO = None
    
    if model_args.usar_mcl_ajustado == True:        
        # Diretório do modelo
        DIRETORIO_MODELO = copiaModeloAjustado()
        
        logging.info("Usando modelo BERT ajustado.")
        
    else:
        DIRETORIO_MODELO = downloadModeloPretreinado(model_args)
        logging.info("Usando modelo BERT pré-treinado.")        
        
    return DIRETORIO_MODELO

## 4.2 Tokenizador BERT

### Função carrega Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

In [ ]:
# Import das bibliotecas.
from transformers import BertTokenizer # Importando as bibliotecas do tokenizador BERT.

def carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args):
    """
      Carrega o tokenizador do DIRETORIO_MODELO.
      O tokenizador utiliza WordPiece.
      Carregando o tokenizador do diretório "./modelo/" do diretório padrão se variável `DIRETORIO_MODELO` setada.
      Caso contrário carrega da comunidade
      Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.
      O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado a partir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.
    
      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
        `model_args` - Objeto com os argumentos do modelo.       
    
      Retorno:
        `tokenizer` - Tokenizador BERT.
    """

    tokenizer = None
    
    # Se a variável DIRETORIO_MODELO foi setada.
    if DIRETORIO_MODELO:
        # Carregando o Tokenizador.
        logging.info("Carregando o tokenizador BERT do diretório {}.".format(DIRETORIO_MODELO))

        tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, do_lower_case=model_args.do_lower_case)

    else:
        # Carregando o Tokenizador da comunidade.
        logging.info("Carregando o tokenizador BERT da comunidade.")

        tokenizer = BertTokenizer.from_pretrained(model_args.pretrained_model_name_or_path, do_lower_case=model_args.do_lower_case)

    return tokenizer

## 4.3 Carrega o modelo e tokenizador BERT

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

### Função carrega modelo BERT medida

In [ ]:
# Import das bibliotecas.
from transformers import BertModel # Importando as bibliotecas do Modelo BERT.

def carregaModeloMedida(DIRETORIO_MODELO, model_args):
    """
      Carrega o modelo e retorna o modelo.
    
      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
        `model_args` - Objeto com os argumentos do modelo.   
    
      Retorno:
        `model` - Um objeto do modelo BERT carregado.
    """

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:        
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para cálculo de medidas.".format(DIRETORIO_MODELO))

        model = BertModel.from_pretrained(DIRETORIO_MODELO,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)
        
    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para cálculo de medidas.".format(model_args.pretrained_model_name_or_path))

        model = BertModel.from_pretrained(model_args.pretrained_model_name_or_path,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    return model

### Função carrega o BERT

In [ ]:
def carregaBERT(model_args):
    """ 
      Carrega o BERT para cálculo de medida ou classificação e retorna o modelo e o tokenizador.
      O tipo do model retornado pode ser BertModel ou BertForSequenceClassification, depende do tipo de model_args.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
          - Se model_args = ModeloArgumentosClassificacao deve ser carregado o BERT para classificação(BertForSequenceClassification).
          - Se model_args = ModeloArgumentosMedida deve ser carregado o BERT para cálculo de medida(BertModel).

      Retorno:    
        `model` - Um objeto do modelo BERT carregado.       
        `tokenizer` - Um objeto tokenizador BERT carregado.       
    """
            
    # Verifica a origem do modelo
    DIRETORIO_MODELO = verificaModelo(model_args)
    
    # Variável para conter o modelo
    model = None
    
    # Carrega o modelo para cálculo da medida
    model = carregaModeloMedida(DIRETORIO_MODELO, model_args)
                
    # Carrega o tokenizador. 
    # O tokenizador é o mesmo para o classificador e medidor.
    tokenizer = carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args)
    
    return model, tokenizer

### Carrega o BERT

In [ ]:
# Carrega o modelo e tokenizador do BERT
model, tokenizer = carregaBERT(model_args)

2022-03-09 12:20:52,617 : INFO : Download do arquivo: bert-large-portuguese-cased_pytorch_checkpoint.zip.


  0%|          | 0.00/1.24G [00:00<?, ?B/s]

2022-03-09 12:22:37,634 : INFO : Download do arquivo: COHEBERT_MANUAL/modeloBERT/vocab.txt.


  0%|          | 0.00/210k [00:00<?, ?B/s]

2022-03-09 12:22:38,377 : INFO : Diretório COHEBERT_MANUAL/modeloBERT do modelo BERT pronta!
2022-03-09 12:22:38,555 : INFO : Usando modelo BERT pré-treinado.
2022-03-09 12:22:38,557 : INFO : Carregando o modelo BERT do diretório COHEBERT_MANUAL/modeloBERT para cálculo de medidas.
2022-03-09 12:22:46,121 : INFO : Carregando o tokenizador BERT do diretório COHEBERT_MANUAL/modeloBERT.


### Recupera detalhes do BERT

In [ ]:
# Verifica o nome do modelo BERT a ser utilizado
MODELO_BERT = getNomeModeloBERT(model_args)

# Verifica o tamanho do modelo(default large)
TAMANHO_BERT = getTamanhoBERT(model_args)

## 4.4 Funções auxiliares do BERT

### getJanelasLista

In [ ]:
def getJanelaSentenca(lista, janela, passo, indice_passo):
  """ 
     Cria janelas de itens de uma lista
         
     Parâmetros:
       `lista` - Um dataframe com os documentos.
       `janela` - Tamanho da janela a ser montada.
       `passo` - Passo ser deslocado na  janela.
       `indice_passo` - Índice do passo que se deseja da janela.

     Retorno:    
       `lista_janela` - Lista com os documentos em janelas.
       `string_janela` - String com os documentos em janelas.
  """
  # Percorre a lista  
  # Dentro do intervalo dos passos
  #if indice_sentenca >= 0 and indice_sentenca < len(lista):
  if indice_passo >= 0 and indice_passo < len(lista):
      # Seleciona o passo que se deseja a janela de 
      #i = indice_passo
      #print("\nPasso :", i)
      
      # Guarda os itens da janela
      lista_janela = []   
      
      # Inicio da lista sem janelas completas antes do passo
      if indice_passo < janela:  

        #print("Inicio da lista") 
        # Sentenças anteriores
        #Evita estourar o início da lista
        inicio = 0
        fim = indice_passo
        for j in range(inicio, fim):                  
          documento_original = lista.iloc[j]
          documento = documento_original['documento']
          lista_janela.append(documento)

        # Sentença central
        documento_original = lista.iloc[indice_passo]
        documento = documento_original['documento']
        lista_janela.append(documento)
        
        # Sentenças posteriores
        inicio = indice_passo + 1
        fim = indice_passo + janela + 1
        for j in range(inicio,fim):        
          documento_original = lista.iloc[j]
          documento = documento_original['documento']
          lista_janela.append(documento)
                    
      else:
        # Meio da lista com janelas completas antes e depois
        if indice_passo < len(lista)-janela:          
          #print(" Meio da lista")

          # Sentenças anteriores
          inicio = indice_passo - janela         
          fim = indice_passo
          for j in range(inicio, fim):          
            documento_original = lista.iloc[j]
            documento = documento_original['documento']
            lista_janela.append(documento)
                        
          # Sentença central
          documento_original = lista.iloc[indice_passo]
          documento = documento_original['documento']
          lista_janela.append(documento)

          # Sentenças posteriores
          inicio = indice_passo + 1
          fim = indice_passo + 1 + janela
          for j in range(inicio,fim):      
            documento_original = lista.iloc[j]
            documento = documento_original['documento']
            lista_janela.append(documento)            

        else:
          # Fim da lista sem janelas completas depois
          if indice_passo >= len(lista)-janela:            
            #print("Fim da lista")    
            
            # Sentenças anteriores
            inicio = i - janela
            fim = i
            for j in range(inicio, fim):
              documento_original = lista.iloc[j]
              documento = documento_original['documento']
              lista_janela.append(documento)
                  
            # Sentença central
            docucmento_original = lista.iloc[indice_passo]
            documento = documento_original['documento']
            lista_janela.append(documento)
                        
            # Sentenças posteriores
            inicio = indice_passo + 1
            fim = indice_passo + 1 + janela 
            # Evita o extrapolar o limite da lista de sentenças
            if fim > len(lista):
              fim = len(lista)
            for j in range(inicio,fim):
              documento_original = lista.iloc[j]
              documento = documento_original['documento']
              lista_janela.append(documento)
                    
  else:
    logging.info("Índice fora do intervalo da lista de passos.")

  return lista_janela, " ".join(lista_janela)

### concatenaListas

In [ ]:
def concatenaListas(lista, pos=1):
  listaConcat = []

  for x in lista:
      listaConcat = listaConcat + x[pos]
  
  return listaConcat

### getEmbeddingsCamadas

Funções que recuperam os embeddings das camadas:
- Primeira camada;
- Penúltima camada;
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingPrimeiraCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][0]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado

def getEmbeddingPenultimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][-2]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado

def getEmbeddingUltimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
     
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][-1]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado    

def getEmbeddingSoma4UltimasCamadas(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  embeddingCamadas = output[2][-4:]
  # Saída: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  

  # Usa o método `stack` para criar uma nova dimensão no tensor 
  # com a concateção dos tensores dos embeddings.        
  #Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultadoStack = torch.stack(embeddingCamadas, dim=0)
  # Saída: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  
  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultadoStack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>
  
  return resultado

def getEmbeddingConcat4UltimasCamadas(output):  
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Cria uma lista com os tensores a serem concatenados
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)  
  # Lista com os tensores a serem concatenados  
  listaConcat = []

  # Percorre os 4 últimos
  for i in [-1,-2,-3,-4]:
      # Concatena da lista
      listaConcat.append(output[2][i])

  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)    
  # Realiza a concatenação dos embeddings de todos as camadas
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)  
  resultado = torch.cat(listaConcat, dim=-1)
  
  # Saída: Entrada: (<1(lote)> x <qtde_tokens> x <3072 ou 4096>)    
  return resultado   

def getEmbeddingSomaTodasAsCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
   
  # Retorna todas as camadas descontando a primeira(0)
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  embeddingCamadas = output[2][1:]
  # Saída: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  # Usa o método `stack` para criar uma nova dimensão no tensor 
  # com a concateção dos tensores dos embeddings.        
  #Entrada: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultadoStack = torch.stack(embeddingCamadas, dim=0)
  # Saída: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
    
  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultadoStack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>
    
  return resultado

### getEmbeddingsVisual

Função para gerar as coordenadas de plotagem a partir das sentenças de embeddings.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingsVisualUltimaCamada(documento, modelo, tokenizer):
    
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualSoma4UltimasCamadas(documento, modelo, tokenizer):
    
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualConcat4UltimasCamadas(documento, modelo, tokenizer):
    
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding        
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualSomaTodasAsCamadas(documento, modelo, tokenizer):
    
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

### getEmbeddings

Função para gerar os embeddings de sentenças.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingsUltimaCamada(documento, modelo, tokenizer):
    
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)
 
    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsSoma4UltimasCamadas(documento, modelo, tokenizer):
    
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)
   
    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsConcat4UltimasCamadas(documento, modelo, tokenizer):
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)
    
    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)
    
    # Camada embedding    
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsConcat4UltimasCamadasJanela(documento, janela_documento, modelo, tokenizer):
    # Adiciona os tokens especiais
    janela_documento_marcado = "[CLS] " + janela_documento + " [SEP]"

    # Divide a sentença em tokens
    janela_documento_tokenizado = tokenizer.tokenize(janela_documento_marcado)
    
    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(janela_documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(janela_documento_tokenizado)
    
    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)
    
    # Camada embedding    
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)
    #print("token_embeddings=", token_embeddings.shape)
    
    # Divide a sentença em tokens para procurar na janela
    documento_tokenizado = tokenizer.tokenize(documento)

    # Procura o local de início e fim do documento na janela
    inicio, fim = encontrarIndiceSubLista(janela_documento_tokenizado, documento_tokenizado)
    #print("Inicio:",inicio," Fim:",fim) 
 
    # Recupera os embeddings dos tokens da sentença a partir dos embeddings do documento
    token_embeddings_retorno = token_embeddings[inicio:fim+1]
    #print("token_embeddings_retorno=", token_embeddings_retorno.shape)

    return token_embeddings_retorno, documento_tokenizado

In [ ]:
def getEmbeddingsSomaTodasAsCamadas(documento, modelo, tokenizer):
    
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

### getDocumentoTokenizado 

Retorna o documento tokenizado

In [ ]:
def getDocumentoTokenizado(documento, tokenizer):
    """
      Retorna o documento tokenizado pelo BERT.
    
      Parâmetros:
      `documento` - Documento a ser tokenizado.
      `tokenizer` - Tokenizador do BERT.
    """    

    # Adiciona os tokens especiais.
    documentoMarcado = "[CLS] " + documento + " [SEP]"

    # Documento tokenizado
    documentoTokenizado = tokenizer.tokenize(documentoMarcado)

    del tokenizer

    return documentoTokenizado    

### encontrarIndiceSubLista 

Retorna os índices de início e fim da sublista na lista

In [ ]:
# Localiza os índices de início e fim de uma sublista em uma lista
def encontrarIndiceSubLista(lista, sublista):

    """
      Localiza os índices de início e fim de uma sublista em uma lista.
    
      Parâmetros:
      `lista` - Uma lista.
      `sublista` - Uma sublista a ser localizada na lista.
    """    
    # https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm

    # Recupera o tamanho da lista 
    h = len(lista)
    # Recupera o tamanho da sublista
    n = len(sublista)    
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    while i < h:
        for j in range(n):
            if lista[i - j] != sublista[-j - 1]:
                i += skip.get(lista[i], n)
                break
        else:
            indiceInicio = i - n + 1
            indiceFim = indiceInicio + len(sublista)-1
            return indiceInicio, indiceFim
    return -1, -1

### getEmbeddingSentencaEmbeddingDocumentoComTodasPalavras

A partir dos embeddings do documento, localiza o indíce de início e fim de uma sentença no documento e retorna os embeddings da sentença.

In [ ]:
def getEmbeddingSentencaEmbeddingDocumentoComTodasPalavras(embeddingDocumento, 
                                                           tokenBERTDocumento, 
                                                           sentenca, 
                                                           tokenizer):

  # Tokeniza a sentença
  sentencaTokenizadaBert = getDocumentoTokenizado(sentenca, tokenizer)
  #print(sentencaTokenizadaBert)

  # Remove os tokens de início e fim da sentença
  sentencaTokenizadaBert.remove("[CLS]")
  sentencaTokenizadaBert.remove("[SEP]")    
  #print(len(sentencaTokenizadaBert))
  
  # Localiza os índices dos tokens da sentença no documento
  inicio, fim = encontrarIndiceSubLista(tokenBERTDocumento, sentencaTokenizadaBert)
  #print(inicio,fim) 
 
  # Recupera os embeddings dos tokens da sentença a partir dos embeddings do documento
  embeddingSentenca = embeddingDocumento[inicio:fim+1]
  #print("embeddingSentenca=", embeddingSentenca.shape)

  del tokenizer
  del tokenBERTDocumento
  del embeddingDocumento
  
  # Retorna o embedding da sentença no documento
  return embeddingSentenca, sentencaTokenizadaBert

### getEmbeddingDocumentoComTodasPalavrasMean

In [ ]:
# Importa a biblioteca
import torch

def getEmbeddingDocumentoComTodasPalavrasMean(embeddingDocumento):
  """
    Calcula a média dos embeddings do documento excluindo os tokens 
    especiais [CLS] do início e [SEP] do fim.
    Remove primeira dimensão devido ao cálculo da média.
    
    Parâmetros:
    `embeddingDocumento` - Embedding do documento.
  """


  # Calcula a média dos embeddings para os tokens de embeddingDocumento, removendo a primeira dimensão.
  # Entrada: <qtde_tokens> x <768 ou 1024>  
  #print("embeddingDocumento1=", embeddingDocumento.shape)
  mediaEmbeddingDocumento = torch.mean(embeddingDocumento[1:-1], dim=0)    
  # Saída: <768 ou 1024>

  del embeddingDocumento

  return mediaEmbeddingDocumento

### getEmbeddingDocumentoRelevanteMean

In [ ]:
# Importa a biblioteca
import torch

def getEmbeddingDocumentoRelevanteMean(id_documento, indexsentenca, embeddingDocumento, tokenBERTDocumento, documento, token_documento, pos_documento, filtro):
  """
    Calcula a média dos embeddings do documento considerando tokens do tipo 
    especificado no filtro
    Remove primeira dimensão devido ao cálculo da média.
    
    Parâmetros:    
    `embeddingDocumento` - Embeddings do documento gerados pelo BERT.
    `tokenBERTDocumento` - Lista com os tokens do documento gerados pelo tokenizador BERT.
    `documento` - Texto com o documento.
    `tokenizer` - Tokenizador do BERT.
    `token_documento` - Lista com os tokens do documento.
    `pos_documento` - Lista com as POS-Tagging do documento.
    `filtro` - Filtro dos embeddings.

  """  
   
  # Recupera a lista de tokens do documento, a lista dos postagging e a lista dos seus embeddings com um mesmo tamanho
  listaTokens, listaPOS, listaEmbeddings = getTokensEmbeddingsPOSSentenca(id_documento, indexsentenca, embeddingDocumento, tokenBERTDocumento, documento, token_documento, pos_documento)

  #print("len(tokenBERTDocumento):", len(tokenBERTDocumento))
  #print("tokenBERTDocumento:", tokenBERTDocumento)
  #print("len(pos_documento):", len(pos_documento))
  #print("pos_documento:", pos_documento)
  #print("filtro:", filtro)
  #print()

  # Lista com os tensores selecionados
  listaTokensSelecionados = []
  # Localizar os embeddings dos tokens da sentença tokenizada sem stop word no documento  
  for i, tokenDocumento in enumerate(listaTokens):     
      if (listaPOS[i] in filtro):          
          #print("Adicionando palavra do embedding:", listaTokens[i])
          listaTokensSelecionados.append(listaEmbeddings[i])

  if  len(listaTokensSelecionados) != 0:
      # Empila os embeddings da lista pela dimensão 0
      embeddingRelevante = torch.stack(listaTokensSelecionados, dim=0)
      #print("embeddingRelevante.shape:",embeddingRelevante.shape)

      # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
      # Entrada: <qtde_tokens> x <768 ou 1024>  
      mediaEmbeddingRelevante = torch.mean(embeddingRelevante, dim=0)    
      # Saída: <768 ou 1024>
      #print("mediaEmbeddingRelevante.shape:", mediaEmbeddingRelevante.shape)
  else:
      mediaEmbeddingRelevante = None

  del embeddingDocumento
  del tokenBERTDocumento
  del documento
  del tokenizer
  del token_documento
  del pos_documento

  return mediaEmbeddingRelevante

### getEmbeddingDocumentoMean

Filtros:
- ALL - Sentença com todas as palavras
- NOUN - Sentença somente com substantivos
- VERB - Sentença somente com verbos
- VERB,NOUN - Sentença somente com verbos e substantivos

In [ ]:
def getEmbeddingDocumentoMean(id_documento, 
                              indexsentenca, 
                              embeddingDocumento, 
                              tokenBERTDocumento, 
                              documento, 
                              tokenizer,
                              token_documento, 
                              pos_documento, 
                              filtro=["ALL"]):  
  """
    Rediciona o cálculo da média dos embeddings de acordo com o filtro especificado.
    
    Parâmetros:    
    `embeddingDocumento` - Embeddings do documento gerados pelo BERT.
    `tokenBERTDocumento` - Lista com os tokens do documento gerados pelo tokenizador BERT.
    `documento` - Texto com o documento.
    `tokenizer` - Tokenizador do BERT.
    `token_documento` - Lista com os tokens do documento.
    `pos_documento` - Lista com as POS-Tagging do documento.
    `filtro` - Filtro dos embeddings.
  """

  if "ALL" in filtro:
    return getEmbeddingDocumentoComTodasPalavrasMean(embeddingDocumento)
  else:
    return getEmbeddingDocumentoRelevanteMean(id_documento, indexsentenca, embeddingDocumento, tokenBERTDocumento, documento, token_documento, pos_documento, filtro)
    

# 5 Comparar Palavras

## 5.1 Carregamento dos arquivos de dados originais 

#### 5.1.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOMEARQUIVOORIGINAL = "original.csv"
NOMEARQUIVOORIGINALCOMPACTADO = "original.zip"
NOMEARQUIVOORIGINALPOS = "originalpos.csv"
NOMEARQUIVOORIGINALPOSCOMPACTADO = "originalpos.zip"

### 5.1.2 Cria o diretório local para receber os dados

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

2022-03-09 12:22:46,795 : INFO : Diretório já existe: /content/COHEBERT_MANUAL.


### 5.1.3 Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOMEARQUIVOORIGINALCOMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOMEARQUIVOORIGINALPOSCOMPACTADO" "$DIRETORIO_LOCAL"
 
  logging.info("Terminei a cópia.")

2022-03-09 12:22:49,213 : INFO : Terminei a cópia.


Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOMEARQUIVOORIGINALCOMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOMEARQUIVOORIGINALPOSCOMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

2022-03-09 12:22:49,702 : INFO : Terminei a descompactação.


### 5.1.4 Carregamento das lista com os dados dos arquivos originais

#### Carrega o arquivo dos dados originais e POS

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOMEARQUIVOORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOMEARQUIVOORIGINALPOS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

2022-03-09 12:22:49,758 : INFO : TERMINADO ORIGINAIS: 40.
2022-03-09 12:22:49,761 : INFO : TERMINADO ORIGINAIS POS: 40.


In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento,classe
11,12,['Como empilhar e desempilhar elementos em uma...,Como empilhar e desempilhar elementos em uma e...,0
18,19,['O que é uma fila e como enfileirar seu eleme...,O que é uma fila e como enfileirar seu elemento?,1
17,18,['O que é uma fila e como empilhar seu element...,O que é uma fila e como empilhar seu elemento?,0
8,9,['Como empilhar elementos em uma estrutura de ...,Como empilhar elementos em uma estrutura de da...,1
16,17,['O que é uma pilha e como empilhar seu elemen...,O que é uma pilha e como empilhar seu elemento?,1


In [ ]:
lista_documentos_originais_pos.sample(5)

,id,pos_documento
12,13,"[[['Como', 'desempilhar', 'elementos', 'em', '..."
24,25,"[[['O', 'que', 'é', 'uma', 'fila', 'e', 'como'..."
10,11,"[[['Como', 'empilhar', 'e', 'desempilhar', 'el..."
31,32,"[[['O', 'que', 'é', 'uma', 'pilha', 'e', 'como..."
9,10,"[[['Como', 'empilhar', 'elementos', 'em', 'uma..."


#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

2022-03-09 12:22:49,876 : INFO : TERMINADO CORREÇÃO ORIGINAIS: 40.
2022-03-09 12:22:49,878 : INFO : TERMINADO CORREÇÃO ORIGINAIS POS: 40.


## 5.2 Gerando as comparações



### 5.2.1 Medidas de similaridade 


Similaridade do cosseno entre os embeddings.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html#scipy.spatial.distance.cosine

A função spatial.distance.cosine do módulo scipy calcula a distância em vez da similaridade do cosseno, mas para conseguir isso, podemos subtrair o valor da distância de 1.

Intervalo de [-1,1] 

Vetores iguais a distância é igual 1.

Vetores diferentes medida próxima de -1.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cosine

def similaridadeCosseno(embeddings1, embeddings2):
    """
      Similaridade do cosseno dos embeddings dos textos.
      
      Parâmetros:
      `embeddings1` - Um embedding a ser medido.
      `embeddings2` - Um embedding a ser medido.
    """
    
    similaridade = 1 - cosine(embeddings1, embeddings2)
    
    return similaridade

### 5.2.2 Medidas de distância 

Distância euclidiana entre os embeddings.

Possui outros nomes como distância L2 ou norma L2.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html#scipy.spatial.distance.euclidean

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import euclidean

def distanciaEuclidiana(embeddings1, embeddings2):
    """
      Distância euclidiana entre os embeddings dos textos.
      Possui outros nomes como distância L2 ou norma L2.
      
      Parâmetros:
      `embeddings1` - Um embedding a ser medido.
      `embeddings2` - Um embedding a ser medido.
    """
    
    distancia = euclidean(embeddings1, embeddings2)
    
    return distancia

Distância Manhattan entre os embeddings.

Possui outros nomes como distância Cityblock, distância L1, norma L1 e métrica do táxi.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cityblock.html#scipy.spatial.distance.cityblock

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cityblock

def distanciaManhattan(embeddings1, embeddings2):
    """
      Distância Manhattan entre os embeddings dos textos 
      Possui outros nomes como distância Cityblock, distância L1, norma L1 e métrica do táxi.
      
      Parâmetros:
      `embeddings1` - Um embedding a ser medido.
      `embeddings2` - Um embedding a ser medido.
    """
    
    distancia = cityblock(embeddings1, embeddings2)

    return distancia

### 5.2.3 Retorna todas as medidas dos embeddings

In [ ]:
def getMedidasEmbedding(embeddingWi, embeddingWj):

  """
    Retorna as medidas de similaridade do cosseno(cos), distância Euclidiana(euc) e 
    distância de Manhattan(man) entre os embeddings.
        
    Parâmetros:
    `embeddingsWi` - Um embedding de uma palavra a ser medido.
    `embeddingsWj` - Um embedding de uma palavra a ser medido.
  """
 
  #print("embeddingWi=", embeddingWi.shape) 
  #print("embeddingWj=", embeddingWj.shape)

  # Similaridade do cosseno entre os embeddings wi e wj
  # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
  cos = similaridadeCosseno(embeddingWi, embeddingWj)
  # Saída: Número real

  # Distância euclidiana entre os embeddings wi e wj
  # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
  euc = distanciaEuclidiana(embeddingWi, embeddingWj)
  # Saída: Número real

  # Distância de manhattan entre os embeddings wi e wj
  # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
  man = distanciaManhattan(embeddingWi, embeddingWj)
  # Saída: Número real

  del embeddingWi
  del embeddingWj
   
  # Retorno das medidas das sentenças
  return cos, euc, man

### 5.2.4 getTokensEmbeddingsPOSSentenca
Gera os tokens, POS e embeddings de cada sentença.

In [ ]:
# Dicionário de tokens de exceções e seus deslocamentos para considerar mais tokens do BERT em relação ao spaCy
# A tokenização do BERT gera mais tokens que a tokenização das palavras do spaCy
dic_excecao_maior = {"":-1,
                    }

In [ ]:
def getExcecaoDicMaior(id, token, dic_excecao_maior):   
    
  valor = dic_excecao_maior.get(token)
  if valor != None:
      return valor
  else:
      return -1

In [ ]:
# Dicionário de tokens de exceções e seus deslocamentos para considerar menos tokens do BERT em relação ao spaCy
# A tokenização do BERT gera menos tokens que a tokenização das palavras do spaCy
dic_excecao_menor = {"1°":1,
                    }

In [ ]:
def getExcecaoDicMenor(id, token, dic_excecao_menor):   
    
  valor = dic_excecao_menor.get(token)
  if valor != None:
      return valor
  else:
      return -1

Função que retorna os embeddings, tokens e POS da sentença com um mesmo tamanho.

In [ ]:
# Importa a biblioteca
import torch

def getTokensEmbeddingsPOSSentenca(id_documento, 
                                   indexsentenca, 
                                   embeddingDocumento, 
                                   tokenBERTDocumento, 
                                   sentenca, 
                                   tokenizer, 
                                   sentenca_token = None, 
                                   sentenca_pos = None, 
                                   estrategia_medida = 0):
    """    
      Retorna os tokens, as postagging e os embeddings dos tokens igualando a quantidade de tokens do spaCy com a tokenização do BERT de acordo com a estratégia. 
      Usa a estratégia MEAN para calcular a média dos embeddings dos tokens que formam uma palavra.
      Usa a estratégia MAX para calcular o valor máximo dos embeddings dos tokens que formam uma palavra.
    """
   
    #Guarda os tokens e embeddings
    listaTokens = []
    listaEmbeddings = []

    if sentenca_token == None:
      # Gera a tokenização e POS-Tagging da sentença    
      sentenca_token, sentenca_pos = getListaTokensPOSSentenca(sentenca)

    #print("\nsentenca            :",sentenca)    
    #print("id_documento        :",id_documento)
    #print("indexsentenca       :",indexsentenca)    
    #print("sentenca_token      :",sentenca_token)
    #print("len(sentenca_token) :",len(sentenca_token))    
    #print("sentenca_pos        :",sentenca_pos)
    #print("len(sentenca_pos)   :",len(sentenca_pos))
    
    # Recupera os embeddings da sentença dos embeddings do documento    
    embeddingSentenca, sentencaTokenizadaBert = getEmbeddingSentencaEmbeddingDocumentoComTodasPalavras(embeddingDocumento, 
                                                                                                       tokenBERTDocumento, 
                                                                                                       sentenca, 
                                                                                                       tokenizer)
    
    # embedding <qtde_tokens x 4096>        
    #print("embeddingSentenca          :",embeddingSentenca.shape)
    #print("sentencaTokenizadaBert     :",sentencaTokenizadaBert)
    #print("len(sentencaTokenizadaBert):",len(sentencaTokenizadaBert))

    # Seleciona os pares de palavra a serem avaliadas
    posWi = 0 # Posição do token da palavra gerado pelo spaCy
    posWj = posWi # Posição do token da palavra gerado pelo BERT

    # Enquanto o indíce da palavra posWj(2a palavra) não chegou ao final da quantidade de tokens do BERT
    while posWj < len(sentencaTokenizadaBert):  

      # Seleciona os tokens da sentença
      Wi = sentenca_token[posWi] # Recupera o token da palavra gerado pelo spaCy
      Wi1 = ""
      if posWi+1 < len(sentenca_token):
        Wi1 = sentenca_token[posWi+1] # Recupera o próximo token da palavra gerado pelo spaCy
  
        # Localiza o deslocamento da exceção
        #pos = getExcecao(Wi,lista_excecao,lista_deslocamento)
        pos2 = getExcecaoDicMenor(id_documento, Wi+Wi1, dic_excecao_menor)  
        #print("Exceção pos2:", pos2)

      Wj = sentencaTokenizadaBert[posWj] # Recupera o token da palavra gerado pelo BERT
      #print("Wi[",posWi,"]=", Wi)
      #print("Wj[",posWj,"]=", Wj)

      # Tratando exceções
      # Localiza o deslocamento da exceção
      #pos = getExcecao(Wi,lista_excecao,lista_deslocamento)
      pos = getExcecaoDicMaior(id_documento, Wi, dic_excecao_maior)  
      #print("Exceção pos:", pos)
            
      if pos != -1 or pos2 != -1:      
        if pos != -1:
          #print("Adiciona 1 Exceção palavra == Wi or palavra = [UNK]:",Wi)
          listaTokens.append(Wi)          
          # Verifica se tem mais de um token
          if pos != 1:
            indiceToken = posWj + pos
            #print("Calcula a média de :", posWj , "até", indiceToken)
            embeddingsTokensPalavra = embeddingSentenca[posWj:indiceToken]
            #print("embeddingsTokensPalavra:",embeddingsTokensPalavra.shape)
            # calcular a média dos embeddings dos tokens do BERT da palavra
            embeddingMedia = torch.mean(embeddingsTokensPalavra, dim=0)
            #print("embeddingMedia:",embeddingMedia.shape)
            listaEmbeddings.append(embeddingMedia)
          else:
            # Adiciona o embedding do token a lista de embeddings
            listaEmbeddings.append(embeddingSentenca[posWj])
         
          # Avança para a próxima palavra e token do BERT
          posWi = posWi + 1
          posWj = posWj + pos
          #print("Proxima:")            
          #print("Wi[",posWi,"]=", sentenca_token[posWi])
          #print("Wj[",posWj,"]=", sentencaTokenizadaBert[posWj])
        else:
          if pos2 != -1:
            #print("Adiciona 1 Exceção palavra == Wi or palavra = [UNK]:",Wi)
            listaTokens.append(Wi+Wi1)          
            # Verifica se tem mais de um token
            if pos2 == 1: 
              # Adiciona o embedding do token a lista de embeddings
              listaEmbeddings.append(embeddingSentenca[posWj])
          
            # Avança para a próxima palavra e token do BERT
            posWi = posWi + 2
            posWj = posWj + pos2
            #print("Proxima:")            
            #print("Wi[",posWi,"]=", sentenca_token[posWi])
            #print("Wj[",posWj,"]=", sentencaTokenizadaBert[posWj])
      else:  
        # Tokens iguais adiciona a lista, o token não possui subtoken
        if (Wi == Wj or Wj=="[UNK]"):
          # Adiciona o token a lista de tokens
          #print("Adiciona 2 Wi==Wj or Wj==[UNK]:", Wi )
          listaTokens.append(Wi)          
          # Adiciona o embedding do token a lista de embeddings
          listaEmbeddings.append(embeddingSentenca[posWj])
          #print("embedding1[posWj]:", embeddingSentenca[posWj].shape)
          # Avança para a próxima palavra e token do BERT
          posWi = posWi + 1
          posWj = posWj + 1   
              
        else:          
          # A palavra foi tokenizada pelo Wordpice com ## ou diferente do spaCy ou desconhecida
          # Inicializa a palavra a ser montada          
          palavraPOS = Wj
          indiceToken = posWj + 1                 
          while  ((palavraPOS != Wi) and indiceToken < len(sentencaTokenizadaBert)):
              if "##" in sentencaTokenizadaBert[indiceToken]:
                # Remove os caracteres "##" do token
                parte = sentencaTokenizadaBert[indiceToken][2:]
              else:                
                parte = sentencaTokenizadaBert[indiceToken]
              
              palavraPOS = palavraPOS + parte
              #print("palavraPOS:",palavraPOS)
              # Avança para o próximo token do BERT
              indiceToken = indiceToken + 1

          #print("\nMontei palavra:",palavraPOS)
          if (palavraPOS == Wi or palavraPOS == "[UNK]"):
              # Adiciona o token a lista
              #print("Adiciona 3 palavra == Wi or palavraPOS = [UNK]:",Wi)
              listaTokens.append(Wi)
              # Calcula a média dos tokens da palavra
              #print("Calcula o máximo :", posWj , "até", indiceToken)
              embeddingsTokensPalavra = embeddingSentenca[posWj:indiceToken]
              #print("embeddingsTokensPalavra2:",embeddingsTokensPalavra)
              #print("embeddingsTokensPalavra2:",embeddingsTokensPalavra.shape)

              # Calcula a média ou máximo dos embeddings das palavras de acordo com a estratégia de pooling dos embeddings
              if estrategia_medida == 0:
                # calcular a média dos embeddings dos tokens do BERT da palavra
                embeddingEstrategia = torch.mean(embeddingsTokensPalavra, dim=0)              
              else:
                # calcular o valor máximo dos embeddings dos tokens do BERT da palavra
                embeddingEstrategia, linha = torch.max(embeddingsTokensPalavra, dim=0)
              
              #print("embeddingEstrategia:",embeddingEstrategia)
              #print("embeddingEstrategia.shape:",embeddingEstrategia.shape)
              listaEmbeddings.append(embeddingEstrategia)

          # Avança para o próximo token do spaCy
          posWi = posWi + 1
          # Pula para o próximo token do BERT
          posWj = indiceToken
    
    # Verificação se as listas estão com o mesmo tamanho
    #if (len(listaTokens) != len(sentenca_token)) or (len(listaEmbeddings) != len(sentenca_token)):
    if (len(listaTokens) !=  len(listaEmbeddings)):
       print("\nsentenca                :",sentenca)  
       print("id_documento            :",id_documento)     
       print("indexsentenca           :",indexsentenca)
       print("sentenca_pos            :",sentenca_pos)
       print("sentenca_token          :",sentenca_token)
       print("sentencaTokenizadaBert  :",sentencaTokenizadaBert)
       print("listaTokens             :",listaTokens)        
       print("len(listaTokens)        :",len(listaTokens))       
       print("listaEmbeddings         :",listaEmbeddings)
       print("len(listaEmbeddings)    :",len(listaEmbeddings))

    del embeddingSentenca
    del tokenBERTDocumento
    del tokenizer

    return listaTokens, sentenca_pos, listaEmbeddings

### 5.2.5 comparaPalavrasSentencaTodas

In [ ]:
def comparaPalavrasSentencaTodas(id_documento, 
                                 indexdocumento, 
                                 indexsentenca, 
                                 embeddingDocumento, 
                                 tokenBERTDocumento, 
                                 sentenca, 
                                 tokenizer, 
                                 sentenca_token = None, 
                                 sentenca_pos = None):

  listaTokens, listaPOS, listaEmbeddings = getTokensEmbeddingsPOSSentenca(id_documento, 
                                                                          indexsentenca, 
                                                                          embeddingDocumento, 
                                                                          tokenBERTDocumento, 
                                                                          sentenca, 
                                                                          tokenizer, 
                                                                          sentenca_token, 
                                                                          sentenca_pos)  
  #print("\nSentença   :",listaTokens)
  #print("POS Tagging:",listaPOS)
  #print("Quantidade de palavras:",len(listaTokens))

  # Quantidade de palavras no documento
  n = len(listaTokens)
  
  # Guarda a comparação da sentença
  lista_comparacao = []
 
  # Realiza o combinação das palavras C(n,p)=(n!/(p!(n-p)!))
  # n = Número de elementos e p as combinações
  # C(5,2) = 10
  # Percorre as palavras da sentença
  for i in range(0,n-1):
  #for i in range(n):
    # Seleciona a palavra i da sentença
    wi = listaTokens[i]
    posi = listaPOS[i]
    #print("i:",i)
    # Percorre as palavras da sentença
    for j in range(i+1,n):
    #for j in range(n):                                
      # Para não comparar a palavra com ela mesma
      #if i != j:
        # Seleciona a palavra j da sentença
        wj = listaTokens[j]
        posj = listaPOS[j]
        
        # Recupera as medidas dos embeddings das palavras
        cos, euc, man = getMedidasEmbedding(listaEmbeddings[i], listaEmbeddings[j])

        # Agrupa as medidas e guarda na lista
        comparacao = [id_documento, indexdocumento, indexsentenca, str(wi), posi, str(wj), posj, cos, euc, man]
        lista_comparacao.append(comparacao)

        #print(comparacao)
        # print("Compara :", i, " com ", j)
        #print("Compara :", wi, " com ", wj)
        # print("     cos:", cos)
        # print("     euc:", euc)
        # print("     man:", man)
  
  return lista_comparacao

### 5.2.6 Realiza a comparação de todas as palavras dos documentos

In [ ]:
# Import das bibliotecas
from tqdm.notebook import tqdm as tqdm_notebook

logging.info("Processando {} documentos originais.".format(len(lista_documentos_originais)))

# Guarda a comparacação das sentenças
resultado_comparacao = []

# Conta sentenças comparadas e não comparadas
contaSentenca = 0

# Barra de progresso dos documentos
lista_documentos_bar = tqdm_notebook(lista_documentos_originais.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_documentos_originais))

# Percorre os documentos
for i, linha_documento in lista_documentos_bar:  
  #if i <  5:     
    #print("linha_documento:",linha_documento)
    # Recupera o id do documento
    id_documento = linha_documento[0]     
    #print("id_documento:",id_documento)    
    # Recupera o documento 
    documento = linha_documento[2]
    #print("documento:",documento) 
    
    # Carrega as listas das sentenças e postagging dos documento    
    lista_sentenca_documento = linha_documento[1]
    # Localiza a POSTagging do documento
    lista_pos_documento = lista_documentos_originais_pos.iloc[i][1]
    #print("lista_sentenca_documento:",lista_sentenca_documento)
    #print("len(lista_sentenca_documento):",len(lista_sentenca_documento))
    #print("lista_pos_documento:",lista_pos_documento)
    #print("len(lista_pos_documento):",len(lista_pos_documento))
        
    # Verifica se é necessário utilizar janelas de documentos 
    if model_args.janela != 0:
      # Recupera os n documentos antes e depois do documento para formar a janela
      lista_Janela_documento, string_janela_documento =  getJanelaSentenca(lista_documentos_originais, 
                                                                          model_args.janela, 
                                                                          i)
      # Submete o documento e a janela de documentos ao BERT para recuperar os embeddings do documento
      # Gera os embeddings do documento utiliza a concatenação das 4 últimas camadas
      embeddingDocumento, tokenBERTDocumento = getEmbeddingsConcat4UltimasCamadasJanela(documento, 
                                                                                          string_janela_documento,
                                                                                          model,                                                                                          
                                                                                          tokenizer)      
                
    else:
      # Gera os embeddings do documento utiliza a concatenação das 4 últimas camadas sem janela
      embeddingDocumento, tokenBERTDocumento = getEmbeddingsConcat4UltimasCamadas(documento, 
                                                                                    model, 
                                                                                    tokenizer)  
      
    # embedding <qtde_tokens x 4096>                
    #print("embeddingDocumento:",embeddingDocumento.shape)
    #print("tokenBERTDocumento:",tokenBERTDocumento)
    #print("len(tokenBERTDocumento):",len(tokenBERTDocumento))
                                             
    # Percorre as sentenças do documento
    for j, sentenca in enumerate(lista_sentenca_documento):      
      #print("id_documento:",id_documento)
      #print("sentenca:",sentenca)

      # Carrega as POSTagging da sentença
      sentenca_token = lista_pos_documento[j][0]
      sentenca_pos = lista_pos_documento[j][1]
      sentenca_verbos = lista_pos_documento[j][2]
      
      #print("sentenca_token:",sentenca_token)
      #print("len(sentenca_token):",len(sentenca_token))

      #print("sentenca_pos:",sentenca_pos)
      #print("len(sentenca_pos):",len(sentenca_pos))

      #print("sentenca_verbos:",sentenca_verbos)
      #print("len(sentenca_verbos):",len(sentenca_verbos))

      # Conta o número de sentenças com palavras comparadas
      contaSentenca = contaSentenca + 1

      # Recupera as maiores e menores medidas entre as palavras
      lista_comparacao = comparaPalavrasSentencaTodas(id_documento, 
                                                      i, 
                                                      j, 
                                                      embeddingDocumento, 
                                                      tokenBERTDocumento, 
                                                      sentenca, 
                                                      tokenizer, 
                                                      sentenca_token, 
                                                      sentenca_pos)
      #print(len(lista_comparacao))
      #print(lista_comparacao)

      # Guarda o resultado da comparação
      resultado_comparacao = resultado_comparacao + lista_comparacao

2022-03-09 12:22:50,349 : INFO : Processando 40 documentos originais.


Documentos:   0%|          | 0/40 [00:00<?, ? documento/s]

In [ ]:
#logging.info("Número de sentenças com palavras comparadas: {}.".format(contaSentenca))
logging.info("Número de comparações                      : {}.".format(len(resultado_comparacao)))

2022-03-09 12:23:44,275 : INFO : Número de comparações                      : 2240.


## 5.3 Gera arquivo das comparações

### 5.3.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOMEARQUIVOCOMPARACAO = "comparacao_palavra_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOMEARQUIVOCOMPARACAOCOMPACTADO = "comparacao_palavra_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.3.2 Gera arquivo comparação

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
dfresultado_comparacao = pd.DataFrame(resultado_comparacao, columns = ["id_documento", "indexdocumento","indexsentenca", "wi", "posi","wj","posj", "cos", "euc","man"])

# Nome do arquivo original
nomeArquivo = DIRETORIO_LOCAL + NOMEARQUIVOCOMPARACAO

# Salva o arquivo original
dfresultado_comparacao.to_csv(nomeArquivo,  sep=";", index=False)

### 5.3.3 Carrega os dados

Carrega os dados das sentencas a partir dos arquivos.


In [ ]:
# Importa das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
dresultado_comparacao = pd.read_csv(DIRETORIO_LOCAL + NOMEARQUIVOCOMPARACAO, sep=";", encoding="UTF-8")

logging.info("Quantidade de comparações: {}.".format(len(dresultado_comparacao)))

2022-03-09 12:23:44,372 : INFO : Quantidade de comparações: 2240.


In [ ]:
dresultado_comparacao.sample(5)

,id_documento,indexdocumento,indexsentenca,wi,posi,wj,posj,cos,euc,man
1613,33,32,0,Como,ADV,implementadas,VERB,0.511271,39.849865,1865.1534
666,19,18,0,O,PRON,e,CCONJ,0.539752,41.447109,2079.2085
2194,40,39,0,uma,DET,?,PUNCT,0.553896,38.102005,1896.1267
431,13,12,0,elementos,NOUN,em,ADP,0.549378,40.671204,1869.5812
1122,26,25,0,que,PRON,?,PUNCT,0.485957,43.765354,2185.5225


### 5.4.4 Compacta e copia o arquivo para uma pasta do GoogleDrive

Compacta o arquivo gerado da comparação para facilitar o envio para o GoogleDrive

Compacta o arquivo.

Usa o zip para compactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 

In [ ]:
!zip -o -j -q "$DIRETORIO_LOCAL$NOMEARQUIVOCOMPARACAOCOMPACTADO" "$DIRETORIO_LOCAL$NOMEARQUIVOCOMPARACAO"

Copia o arquivo  para o GoogleDrive

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
   
    # Copia o arquivo das comparações para o google drive
    #!cp "$DIRETORIO_LOCAL$NOMEARQUIVOCOMPARACAOCOMPACTADO" "$DIRETORIO_DRIVE"
    
    logging.info("Terminei a cópia do arquivo.")

2022-03-09 12:23:44,718 : INFO : Terminei a cópia do arquivo.


# 6 Finalização

## 6.1 Tempo final de processamento



In [ ]:
# Pega o tempo atual menos o tempo do início do processamento.
finalProcessamento = time.time()
tempoTotalProcessamento = formataTempo(finalProcessamento - inicioProcessamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempoTotalProcessamento))


  Tempo processamento:  0:06:12 (h:mm:ss)
